In [ ]:
# 스파크 세션 생성
import pyspark
from pyspark.sql import SparkSession

# SynapseML : 마이크로소프트에서 만든 스파크용 ML 라이브러리
# https://microsoft.github.io/SynapseML/
# config에서 synapseml 모듈 불러오기
spark = SparkSession.builder \
    .appName("Spark study - 231226") \
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.2") \
    .getOrCreate()

In [ ]:
# 복잡한 JSON 파일을 읽을 때는 multiline 옵션을 쓰자

df_bike = spark.read.option('multiline', 'true').json('file:///home/kjh/data/bike/bike.json')
df_weather = spark.read.option('multiline', 'true').json('file:///home/kjh/data/bike/json_cycle_finale.json')

In [3]:
# 파일의 구조 확인
df_bike.printSchema()
df_weather.printSchema()

root
 |-- useStatus: struct (nullable = true)
 |    |-- RESULT: struct (nullable = true)
 |    |    |-- CODE: string (nullable = true)
 |    |    |-- MESSAGE: string (nullable = true)
 |    |-- list_total_count: string (nullable = true)
 |    |-- row: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- END_INDEX: long (nullable = true)
 |    |    |    |-- RENT_CNT: string (nullable = true)
 |    |    |    |-- RENT_NM: string (nullable = true)
 |    |    |    |-- RTN_CNT: string (nullable = true)
 |    |    |    |-- START_INDEX: long (nullable = true)
 |    |    |    |-- STAT_DATA: string (nullable = true)
 |    |    |    |-- STA_LOC: string (nullable = true)

root
 |-- response: struct (nullable = true)
 |    |-- body: struct (nullable = true)
 |    |    |-- dataType: string (nullable = true)
 |    |    |-- items: struct (nullable = true)
 |    |    |    |-- item: array (nullable = true)
 |    |    |    |    |-- element: struct (containsNull

In [4]:
from pyspark.sql.functions import explode

# 중첩된 JSON 구조 접근 및 explode 함수 사용
df_bike = df_bike.select(explode("useStatus.row").alias("items"))
df_weather = df_weather.select(explode("response.body.items.item").alias("items"))

In [5]:
df_bike.show()
df_weather.show()

+-------------------------+
|                    items|
+-------------------------+
| {0, 16, 729. 서부식자...|
|  {0, 20, 731. 서울시 ...|
|{0, 9, 732. 신월중학교...|
|{0, 6, 733. 신정이펜하...|
| {0, 13, 734. 신트리공...|
| {0, 11, 735. 영도초등...|
|{0, 8, 736. 오솔길공원...|
| {0, 5, 737. 장수공원,...|
| {0, 18, 504. 신자초교...|
| {0, 12, 739. 신월사거...|
| {0, 2, 740. 으뜸공원,...|
| {0, 0, 741. 화곡로 입...|
|  {0, 3, 742. 등촌역 5...|
| {0, 3, 743. 현대6차아...|
| {0, 8, 744. 신목동역 ...|
|{0, 3, 745. 강서초등학...|
|  {0, 21, 746. 목동2단...|
|  {0, 15, 747. 목동3단...|
|{0, 3, 748. 목동운동장...|
| {0, 38, 505. 자양사거...|
+-------------------------+
only showing top 20 rows



+--------------------+
|               items|
+--------------------+
|{0.5, 1.9, 2.9, -...|
|{0.7, 1.7, 2.6, 0...|
|{1.0, 1.8, 2.7, 0...|
|{1.1, 1.9, 2.7, 0...|
|{1.0, 1.8, 2.6, 0...|
|{1.3, 1.9, 2.6, 1...|
|{3.8, 3.3, 3.6, 4...|
|{4.6, 4.5, 4.7, 4...|
|{2.9, 3.6, 4.3, 2...|
|{1.8, 2.8, 3.6, 0...|
|{1.6, 2.4, 3.2, 0...|
|{1.6, 2.3, 3.0, 1...|
|{1.1, 2.0, 2.9, 0...|
|{0.5, 1.6, 2.5, -...|
|{0.2, 1.2, 2.1, -...|
|{0.1, 1.0, 1.9, -...|
|{0.1, 1.0, 1.8, -...|
|{0.1, 1.0, 1.8, -...|
|{0.2, 1.0, 1.8, 0...|
|{0.5, 1.2, 1.9, 0...|
+--------------------+
only showing top 20 rows



In [6]:
# items 컬럼의 필드를 별도의 컬럼으로 펼침
df_bike = df_bike.select("items.*")
df_weather = df_weather.select("items.*")

In [7]:
df_bike.createOrReplaceTempView('bike')
df_weather.createOrReplaceTempView('weather')

In [8]:
df_bike.show()

+---------+--------+-------------------------------+-------+-----------+----------+-------+
|END_INDEX|RENT_CNT|                        RENT_NM|RTN_CNT|START_INDEX| STAT_DATA|STA_LOC|
+---------+--------+-------------------------------+-------+-----------+----------+-------+
|        0|      16|     729. 서부식자재마트 건너편|     12|          0|2020-01-01| 양천구|
|        0|      20|   731. 서울시 도로환경관리센터|     20|          0|2020-01-01| 양천구|
|        0|       9|                732. 신월중학교|      2|          0|2020-01-01| 양천구|
|        0|       6|       733. 신정이펜하우스314동|      4|          0|2020-01-01| 양천구|
|        0|      13|           734. 신트리공원 입구|     16|          0|2020-01-01| 양천구|
|        0|      11|              735. 영도초등학교|     17|          0|2020-01-01| 양천구|
|        0|       8|                736. 오솔길공원|      9|          0|2020-01-01| 양천구|
|        0|       5|                  737. 장수공원|      5|          0|2020-01-01| 양천구|
|        0|      18|        504. 신자초교입구교차로|     21|          0|2020-

In [9]:
# 필요없어 보이는 컬럼 삭제
df_bike = spark.sql("""
                    SELECT 
                        RENT_NM, 
                        CAST(RENT_CNT AS INT) AS RENT_CNT, 
                        CAST(RTN_CNT AS INT) AS RTN_CNT, 
                        unix_timestamp(STAT_DATA, 'yyyy-MM-dd') AS STAT_DATA
                    FROM bike
                    """)
df_weather = df_weather.drop('iscs', 'stnNm')

In [10]:
# 컬럼의 형식변환
# tm 컬럼은 String으로 된 YYYY-MM-DD 날짜 형식. VectorAssembler 을 사용하기 위해서는 숫자로 변환 필요
# Date 로 먼저 변환한 뒤, Timestamp 형식으로 변환 => 최종 long 타입

from pyspark.sql.functions import col, unix_timestamp, to_date

df_wt = df_weather

# 'tm' 컬럼을 날짜 타입으로 변환하고, 이를 Unix timestamp로 변환
df_wt = df_wt.withColumn('tm', unix_timestamp(to_date(col('tm'), 'yyyy-MM-dd')))

data_types = {
    "stnId": "int",  # 지점 번호는 정수
    "avgM15Te": "float",  # 지중온도는 소수점을 포함할 수 있으므로 float
    "n99Rn": "float",  # 강수량은 소수점을 포함할 수 있음
    "minPs": "float",  # 기압은 소수점을 포함할 수 있음
    "avgRhm": "float",  # 상대습도는 소수점을 포함할 수 있음
    "minRhmHrmt": "int",  # 시각은 정수로 처리
    "maxInsWsWd": "int",  # 풍향은 정수로 처리
    "avgTs": "float",  # 지면온도는 소수점을 포함할 수 있음
    "maxInsWsHrmt": "int",  # 시각은 정수로 처리
    "ddMesHrmt": "int",  # 시각은 정수로 처리
    "maxPsHrmt": "int",  # 시각은 정수로 처리
    "avgPv": "float",  # 증기압은 소수점을 포함할 수 있음
    "minRhm": "float",  # 상대습도는 소수점을 포함할 수 있음
    "sumSsHr": "float",  # 일조 시간은 소수점을 포함할 수 있음
    "ssDur": "float",  # 가조시간은 소수점을 포함할 수 있음
    "avgPs": "float",  # 해면기압은 소수점을 포함할 수 있음
    "maxWs": "float",  # 풍속은 소수점을 포함할 수 있음
    "avgCm5Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "minTg": "float",  # 초상온도는 소수점을 포함할 수 있음
    "maxWsWd": "int",  # 풍향은 정수로 처리
    "sumSmlEv": "float",  # 증발량은 소수점을 포함할 수 있음
    "avgTca": "float",  # 전운량은 소수점을 포함할 수 있음
    "hr1MaxIcsr": "float",  # 일사량은 소수점을 포함할 수 있음
    "avgTd": "float",  # 이슬점온도는 소수점을 포함할 수 있음
    "maxPs": "float",  # 해면기압은 소수점을 포함할 수 있음
    "avgCm20Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "ddMes": "float",  # 적설량은 소수점을 포함할 수 있음
    "minTa": "float",  # 기온은 소수점을 포함할 수 있음
    "minPsHrmt": "int",  # 시각은 정수로 처리
    "avgM50Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "maxTa": "float",  # 기온은 소수점을 포함할 수 있음
    "hr24SumRws": "int",  # 풍정합은 정수로 처리
    "avgM30Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "avgCm10Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "avgM05Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "hr1MaxIcsrHrmt": "int",  # 시각은 정수로 처리
    "maxInsWs": "float",  # 풍속은 소수점을 포함할 수 있음
    "avgTca": "float",  # 전운량은 소수점을 포함할 수 있음
    "avgCm30Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "avgM10Te": "float",  # 지중온도는 소수점을 포함할 수 있음
    "sumGsr": "float",  # 일사량은 소수점을 포함할 수 있음
    "maxWsHrmt": "int",  # 시각은 정수로 처리
    "avgPa": "float",  # 현지기압은 소수점을 포함할 수 있음
    "avgWs": "float",  # 풍속은 소수점을 포함할 수 있음
    "sumFogDur": "float",  # 안개 지속 시간은 소수점을 포함할 수 있음
    "sumLrgEv": "float",  # 증발량은 소수점을 포함할 수 있음
    "sumDpthFhsc": "float",  # 신적설은 소수점을 포함할 수 있음
    "ddMefs": "float",  # 적설량은 소수점을 포함할 수 있음
    "ddMefsHrmt": "int",  # 시각은 정수로 처리
    "sumRn": "float",  # 강수량은 소수점을 포함할 수 있음
    "hr1MaxRnHrmt": "int",  # 시각은 정수로 처리
    "hr1MaxRn": "float",  # 강수량은 소수점을 포함할 수 있음
    "mi10MaxRnHrmt": "int",  # 시각은 정수로 처리
    "mi10MaxRn": "float",  # 강수량은 소수점을 포함할 수 있음
    "avgTa": "float",  # 기온은 소수점을 포함할 수 있음
    "minTaHrmt": "int",  # 시각은 정수로 처리
    "maxTaHrmt": "int",  # 시각은 정수로 처리
    "maxWd": "int",  # 풍향은 정수로 처리
    "avgLmac": "float",  # 중하층운량은 소수점을 포함할 수 있음
    "sumRnDur": "float"
}

# 딕셔너리 형태의 컬럼 타입정보를 가지고 withColumn을 for문으로 돌림
for column, data_type in data_types.items():
    df_wt = df_wt.withColumn(column, col(column).cast(data_type))

In [11]:
df_bike.printSchema()

root
 |-- RENT_NM: string (nullable = true)
 |-- RENT_CNT: integer (nullable = true)
 |-- RTN_CNT: integer (nullable = true)
 |-- STAT_DATA: long (nullable = true)



In [12]:
# 결측치 컬럼 제거
drop_cols = ['ddMefs', 'ddMefsHrmt', 'ddMes', 'ddMesHrmt', 'hr1MaxRn', 'hr1MaxRnHrmt', 'mi10MaxRn', 'mi10MaxRnHrmt', 'n99Rn', 'sumDpthFhsc', 'sumFogDur', 'sumRn', 'sumRnDur']
df_wt = df_wt.drop(*drop_cols)

# 결측 데이터 있는 행 제거
df_wt = df_wt.dropna()

In [ ]:
# 캐시를 수행하여 아래의 JOIN 작업의 속도를 최적화

df_bike.createOrReplaceTempView('bike')
df_wt.createOrReplaceTempView('wt')
df_bike.cache()
df_wt.cache()

In [14]:
df_fin = spark.sql("""
                   SELECT b.*, w.*
                   FROM bike b
                   INNER JOIN wt w
                   ON b.STAT_DATA = w.tm
                   """)

df_fin = df_fin.drop("tm")

In [15]:
df_fin.show()

+-------------------------------+--------+-------+----------+---------+---------+---------+--------+-------+--------+--------+--------+--------+--------+------+------+-----+------+-----+------+-----+-----+-----+----------+--------------+----------+--------+------------+----------+------+---------+-----+---------+-----+-----+---------+-------+------+---------+------+----------+-----+---------+-----+-----+-----+------+--------+--------+-------+
|                        RENT_NM|RENT_CNT|RTN_CNT| STAT_DATA|avgCm10Te|avgCm20Te|avgCm30Te|avgCm5Te|avgLmac|avgM05Te|avgM10Te|avgM15Te|avgM30Te|avgM50Te| avgPa| avgPs|avgPv|avgRhm|avgTa|avgTca|avgTd|avgTs|avgWs|hr1MaxIcsr|hr1MaxIcsrHrmt|hr24SumRws|maxInsWs|maxInsWsHrmt|maxInsWsWd| maxPs|maxPsHrmt|maxTa|maxTaHrmt|maxWd|maxWs|maxWsHrmt|maxWsWd| minPs|minPsHrmt|minRhm|minRhmHrmt|minTa|minTaHrmt|minTg|ssDur|stnId|sumGsr|sumLrgEv|sumSmlEv|sumSsHr|
+-------------------------------+--------+-------+----------+---------+---------+---------+--------+------

### Pipeline 모듈을 사용하여 라벨 인코딩 + 원핫인코딩 + 벡터어셈블러 한번에 수행

In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import regexp_replace

# 예시: 'RENT_NM' 컬럼의 특수 문자를 '_'로 대체
df_fin = df_fin.withColumn('RENT_NM', regexp_replace('RENT_NM', "[ ,:;{}()\[\]\\n]", "_"))

# StringIndexer를 사용하여 범주형 컬럼을 숫자형으로 변환
stringIndexer = StringIndexer(inputCol="RENT_NM", outputCol="indexed_RENT_NM")

# OneHotEncoder를 사용하여 숫자 인덱스를 원-핫 인코딩된 벡터로 변환
oneHotEncoder = OneHotEncoder(inputCols=["indexed_RENT_NM"], outputCols=["encoded_RENT_NM"])

# 데이터프레임의 모든 컬럼을 특성으로 사용하고자 할 때
features = df_fin.columns

# 라벨 컬럼과 원본 범주형 컬럼 제외
features.remove('RENT_CNT')
features.remove('RENT_NM')

# StringIndexer와 OneHotEncoder의 출력 컬럼 추가
features.append('encoded_RENT_NM')

# VectorAssembler 생성
assembler = VectorAssembler(
    inputCols=features,
    outputCol="features"
)

# 파이프라인 생성 및 모델 학습
pipeline = Pipeline(stages=[stringIndexer, oneHotEncoder, assembler])
transformer = pipeline.fit(df_fin)
tf_df = transformer.transform(df_fin).select('RENT_CNT', 'features')

In [17]:
tf_df.show()

+--------+--------------------+
|RENT_CNT|            features|
+--------+--------------------+
|      16|(2186,[0,1,2,3,4,...|
|      20|(2186,[0,1,2,3,4,...|
|       9|(2186,[0,1,2,3,4,...|
|       6|(2186,[0,1,2,3,4,...|
|      13|(2186,[0,1,2,3,4,...|
|      11|(2186,[0,1,2,3,4,...|
|       8|(2186,[0,1,2,3,4,...|
|       5|(2186,[0,1,2,3,4,...|
|      18|(2186,[0,1,2,3,4,...|
|      12|(2186,[0,1,2,3,4,...|
|       2|(2186,[0,1,2,3,4,...|
|       0|(2186,[0,1,2,3,4,...|
|       3|(2186,[0,1,2,3,4,...|
|       3|(2186,[0,1,2,3,4,...|
|       8|(2186,[0,1,2,3,4,...|
|       3|(2186,[0,1,2,3,4,...|
|      21|(2186,[0,1,2,3,4,...|
|      15|(2186,[0,1,2,3,4,...|
|       3|(2186,[0,1,2,3,4,...|
|      38|(2186,[0,1,2,3,4,...|
+--------+--------------------+
only showing top 20 rows



In [18]:
# train, test 쪼개기
train, test = tf_df.randomSplit([0.7, 0.3], seed=42)

In [19]:
from synapse.ml.lightgbm import LightGBMRegressor

# 모델 생성 및 훈련
lgbm = LightGBMRegressor(objective='regression',
                            featuresCol='features',
                            labelCol='RENT_CNT',
                            alpha=0.3,
                            learningRate=0.3,
                            numIterations=100,
                            numLeaves=31)

model = lgbm.fit(train)

[LightGBM] [Warning] Using too small ``bin_construct_sample_cnt`` may encounter unexpected errors and poor accuracy.
[LightGBM] [Info] Saving data reference to binary buffer


In [20]:
# 예측
pred = model.transform(test)

In [21]:
# predictionCol 확인
model.getPredictionCol()

'prediction'

In [22]:
from pyspark.ml.evaluation import RegressionEvaluator

# 검증
evaluator = RegressionEvaluator(labelCol="RENT_CNT", predictionCol="prediction", metricName="rmse")
# evaluator = RegressionEvaluator(labelCol="RENT_CNT", predictionCol="prediction", metricName="r2")

rmse = evaluator.evaluate(pred)
# r2 = evaluator.evaluate(pred)

23/12/28 04:07:14 WARN DAGScheduler: Broadcasting large task binary with size 1082.2 KiB


In [23]:
rmse
# r2

15.765189308962714

### SynapseML의 AutoML 모듈 사용하여 최적 모델 찾기

In [24]:
from synapse.ml.automl import *
from pyspark.ml.regression import RandomForestRegressor
from synapse.ml.train import TrainRegressor

# 알고리즘 설정
rf = RandomForestRegressor()
lgbm = LightGBMRegressor()

smlmodels = [rf, lgbm]
mmlmodels = [TrainRegressor(model=model, labelCol="RENT_CNT") for model in smlmodels]

In [25]:
# 각 알고리즘의 하이퍼파라미터 설정
paramBuilder  = (
    HyperparamBuilder() 
    .addHyperparam(rf, rf.numTrees, DiscreteHyperParam([10, 20])) 
    .addHyperparam(rf, rf.maxDepth, DiscreteHyperParam([5, 10])) 
    .addHyperparam(lgbm, lgbm.learningRate, RangeHyperParam(0.1, 0.3)) 
    .addHyperparam(lgbm, lgbm.numLeaves, DiscreteHyperParam([31, 50])) 
)

searchSpace = paramBuilder.build()
randomSpace = RandomSpace(searchSpace)
print(searchSpace)
print(randomSpace)

dict_items([(Param(parent='RandomForestRegressor_a8d4df21cd48', name='numTrees', doc='Number of trees to train (>= 1).'), (RandomForestRegressor_a8d4df21cd48, <synapse.ml.automl.HyperparamBuilder.DiscreteHyperParam object at 0x7fb86dd94650>)), (Param(parent='RandomForestRegressor_a8d4df21cd48', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes. Must be in range [0, 30].'), (RandomForestRegressor_a8d4df21cd48, <synapse.ml.automl.HyperparamBuilder.DiscreteHyperParam object at 0x7fb8c410bc50>)), (Param(parent='LightGBMRegressor_acfd2dba4d23', name='learningRate', doc='Learning rate or shrinkage rate'), (LightGBMRegressor_acfd2dba4d23, <synapse.ml.automl.HyperparamBuilder.RangeHyperParam object at 0x7fb86dfd8bc0>)), (Param(parent='LightGBMRegressor_acfd2dba4d23', name='numLeaves', doc='Number of leaves'), (LightGBMRegressor_acfd2dba4d23, <synapse.ml.automl.HyperparamBuilder.DiscreteHyperParam object at 0x7f

In [26]:
# AutoML 설정
automl = TuneHyperparameters(
    evaluationMetric="rmse",            ## 검증방식 설정
    models=mmlmodels,                   ## 모델 알고리즘 설정
    numFolds=2,                         ## 교차 검증 폴드 수
    numRuns=len(mmlmodels),             ## 랜덤 서치 횟수
    parallelism=1,                      ## 병렬로 실행할 모델의 수
    paramSpace=randomSpace.space(),     ## 하이퍼파라미터 공간을 정의하는 객체
    seed=42                             ## 난수 로직 설정
)

# 모델 학습
fittedModel = automl.fit(train)

23/12/28 04:07:28 WARN DAGScheduler: Broadcasting large task binary with size 1045.3 KiB
23/12/28 04:07:47 WARN DAGScheduler: Broadcasting large task binary with size 1109.0 KiB
23/12/28 04:07:58 WARN DAGScheduler: Broadcasting large task binary with size 1235.1 KiB
23/12/28 04:08:10 WARN DAGScheduler: Broadcasting large task binary with size 1486.9 KiB
23/12/28 04:08:24 WARN DAGScheduler: Broadcasting large task binary with size 1990.1 KiB
23/12/28 04:08:39 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
23/12/28 04:08:58 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
23/12/28 04:09:19 WARN DAGScheduler: Broadcasting large task binary with size 7.7 MiB
23/12/28 04:09:45 WARN DAGScheduler: Broadcasting large task binary with size 12.2 MiB
23/12/28 04:10:14 WARN DAGScheduler: Broadcasting large task binary with size 18.4 MiB


[LightGBM] [Info] Saving data reference to binary buffer


23/12/28 04:10:58 WARN DAGScheduler: Broadcasting large task binary with size 1200.5 KiB
23/12/28 04:11:05 WARN DAGScheduler: Broadcasting large task binary with size 1286.4 KiB
23/12/28 04:11:08 WARN DAGScheduler: Broadcasting large task binary with size 1294.7 KiB
23/12/28 04:11:15 WARN DAGScheduler: Broadcasting large task binary with size 1045.4 KiB
23/12/28 04:11:34 WARN DAGScheduler: Broadcasting large task binary with size 1109.0 KiB
23/12/28 04:11:44 WARN DAGScheduler: Broadcasting large task binary with size 1235.1 KiB
23/12/28 04:11:56 WARN DAGScheduler: Broadcasting large task binary with size 1486.9 KiB
23/12/28 04:12:11 WARN DAGScheduler: Broadcasting large task binary with size 1990.0 KiB
23/12/28 04:12:27 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
23/12/28 04:12:46 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
23/12/28 04:13:08 WARN DAGScheduler: Broadcasting large task binary with size 7.6 MiB
23/12/28 04:13:34 WARN DAGSche

In [28]:
# 최적화된 모델의 세부 정보
fittedModel.getBestModelInfo()

'bootstrap: true, cacheNodeIds: false, checkpointInterval: 10, featureSubsetStrategy: auto, featuresCol: TrainRegressor_6a97b1b2d846_features, impurity: variance, labelCol: RENT_CNT, leafCol: , maxBins: 32, maxDepth: 10, maxMemoryInMB: 256, minInfoGain: 0.0, minInstancesPerNode: 1, minWeightFractionPerNode: 0.0, numTrees: 20, predictionCol: prediction, seed: 8246333188144965666, subsamplingRate: 1.0'

In [29]:
# 최적화된 모델 보기
fittedModel.getBestModel()

TrainedRegressorModel_522811e06210

In [33]:
# 최적화된 모델 저장
bestModel = fittedModel.getBestModel()

In [34]:
from synapse.ml.train import ComputeModelStatistics

prediction = bestModel.transform(test)

# ComputeModelStatistics를 사용하여 검증
metrics = ComputeModelStatistics().transform(prediction)
metrics.limit(10).toPandas()

,mean_squared_error,root_mean_squared_error,R^2,mean_absolute_error
0,139.90008,11.827936,0.921076,6.40082


In [35]:
# 검증
evaluator_automl = RegressionEvaluator(labelCol="RENT_CNT", predictionCol="prediction", metricName="rmse")
# evaluator_automl = RegressionEvaluator(labelCol="RENT_CNT", predictionCol="prediction", metricName="r2")

rmse_automl = evaluator_automl.evaluate(prediction)
# r2_automl = evaluator.evaluate(prediction)

In [36]:
rmse_automl

11.82793642320538